<a href="https://colab.research.google.com/github/ITU-Business-Analytics-Team/Business_Analytics_for_Professionals/blob/main/Part%20I%20%3A%20Methods%20%26%20Technologies%20for%20Business%20Analytics/Chapter%206%3A%20Feature%20Engineering/6_2_Dimensionality_Reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Feature Engineering**
## Dimensionality Reduction


[Dataset](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)): from UCI Machine Learning Repository

In [ ]:
import time
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# read the data  CreditRiskGerman.csv
url=   'https://drive.google.com/file/d/1FZjwmptUhgjgNTLUAxkMkrWKWT9vkPgv/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path)

In [ ]:
df

,Account Balance,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,Guarantors,Duration in Current address,Most valuable available asset,Age (years),Concurrent Credits,Type of apartment,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker,Creditability
0,1,18,4,2,1049,1,2,4,2,1,4,2,21,3,1,1,3,1,1,1,1
1,1,9,4,0,2799,1,3,2,3,1,2,1,36,3,1,2,3,2,1,1,1
2,2,12,2,9,841,2,4,2,2,1,4,1,23,3,1,1,2,1,1,1,1
3,1,12,4,0,2122,1,3,3,3,1,2,1,39,3,1,2,2,2,1,2,1
4,1,12,4,0,2171,1,3,4,3,1,4,2,38,1,2,2,2,1,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,24,2,3,1987,1,3,2,3,1,4,1,21,3,1,1,2,2,1,1,0
996,1,24,2,0,2303,1,5,4,3,2,1,1,45,3,2,1,3,1,1,1,0
997,4,21,4,0,12680,5,5,4,3,1,4,4,30,3,3,1,4,1,2,1,0
998,2,12,2,3,6468,5,1,2,3,1,1,4,52,3,2,1,4,1,2,1,0


In [ ]:
df.rename(columns = {'Creditability' : 'class'}, inplace = True)

In [ ]:
# read the data  CreditRisk.csv

X = df.drop(['class'], axis = 1)
Y = df['class']



In [ ]:
X.head()

,Account Balance,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,Guarantors,Duration in Current address,Most valuable available asset,Age (years),Concurrent Credits,Type of apartment,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker
0,1,18,4,2,1049,1,2,4,2,1,4,2,21,3,1,1,3,1,1,1
1,1,9,4,0,2799,1,3,2,3,1,2,1,36,3,1,2,3,2,1,1
2,2,12,2,9,841,2,4,2,2,1,4,1,23,3,1,1,2,1,1,1
3,1,12,4,0,2122,1,3,3,3,1,2,1,39,3,1,2,2,2,1,2
4,1,12,4,0,2171,1,3,4,3,1,4,2,38,1,2,2,2,1,1,2


In [ ]:
X.shape

(1000, 20)

In [ ]:
def forest_test(X, Y):
    X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, 
                                                        test_size = 0.30, 
                                                        random_state = 101)
    start = time.process_time()
    trainedforest = RandomForestClassifier(n_estimators=700).fit(X_Train,Y_Train)
    print(time.process_time() - start)
    predictionforest = trainedforest.predict(X_Test)
    print(confusion_matrix(Y_Test,predictionforest))
    print(classification_report(Y_Test,predictionforest))

In [ ]:
forest_test(X, Y)

1.17283999
[[ 37  53]
 [ 20 190]]
              precision    recall  f1-score   support

           0       0.65      0.41      0.50        90
           1       0.78      0.90      0.84       210

    accuracy                           0.76       300
   macro avg       0.72      0.66      0.67       300
weighted avg       0.74      0.76      0.74       300



## **Principal Component Analysis (PCA)**


PCA is an unsupervised technique that generates linear combinations of the original features. The objective is to extract critical information from the data and express it as a collection of summary features called principal components


In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=8,svd_solver='full')
X_pca = pca.fit_transform(X)
print(pca.explained_variance_)

forest_test(X_pca, Y)

[7.96798475e+06 1.30509160e+02 8.74697259e+01 7.37467891e+00
 2.72449923e+00 1.60783105e+00 1.51627550e+00 1.11772570e+00]
1.4642333479999996
[[ 40  50]
 [ 22 188]]
              precision    recall  f1-score   support

           0       0.65      0.44      0.53        90
           1       0.79      0.90      0.84       210

    accuracy                           0.76       300
   macro avg       0.72      0.67      0.68       300
weighted avg       0.75      0.76      0.75       300



## **Linear Discriminant Analysis (LDA)**



LDA is another linear transformation technique as PCA used for dimension reduction. However, LDA is a supervised and uses class label information.  As PCA represent the direction of maximum variance in data, LDA describes the direction that maximizes class separability. 

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(n_components=1)

# run an LDA and use it to transform the features
X_lda = lda.fit(X, Y).transform(X)

In [ ]:
forest_test(X_lda, Y)

1.192435475
[[ 49  41]
 [ 54 156]]
              precision    recall  f1-score   support

           0       0.48      0.54      0.51        90
           1       0.79      0.74      0.77       210

    accuracy                           0.68       300
   macro avg       0.63      0.64      0.64       300
weighted avg       0.70      0.68      0.69       300



## **AutoEncoder(AE)**


AutoEncoder is a member of ML algorithms which is used as a dimensionality reduction technique. The primary distinction between Autoencoders and other dimensionality reduction approaches is that Autoencoders project data from a high dimension to a lower dimension via non-linear transformations. 

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

input_layer = Input(shape=(X.shape[1],))
encoded = Dense(8, activation='relu')(input_layer)
decoded = Dense(X.shape[1], activation='softmax')(encoded)
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

X1, X2, Y1, Y2 = train_test_split(X, X, test_size=0.3, random_state=101)

autoencoder.fit(X1, Y1,
                epochs=100,
                batch_size=200,
                shuffle=True,
                verbose = 30,
                validation_data=(X2, Y2))

encoder = Model(input_layer, encoded)
X_ae = encoder.predict(X)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [ ]:
forest_test(X_ae, Y)

1.1944744419999989
[[ 31  59]
 [ 47 163]]
              precision    recall  f1-score   support

           0       0.40      0.34      0.37        90
           1       0.73      0.78      0.75       210

    accuracy                           0.65       300
   macro avg       0.57      0.56      0.56       300
weighted avg       0.63      0.65      0.64       300



In this case, the objective is to try to predict if there is a credit risk or not by looking at the given features. The input data is divided into train (%70) and test sets (%30). Random Forest classifier is used to predict the output on the test data using dimension reduction techniques.   As shown, training a Random Forest classifier using PCA (n=8), led to 77% Accuracy and produced better results comparing to other methods LDA and AE. 